In [1]:
!pip install mysql.connector

  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp311-cp311-win_amd64.whl size=247958 sha256=0fba18bd22a5a5549eb4921c8c9dc9836aab8bc20b7de3ff6ef9ea97c0c00a84
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\17\cd\ed\2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql.connector


In [2]:
import face_recognition
import os
import cv2
import mysql.connector
from datetime import datetime

# In[8]:


known_faces_dir1 = r'missingPersonPic'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = "cnn"
known_faces=[]
known_names=[]
location = "RCPIT"

def dataAdd(fname):
    #establishing the connection
    conn = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='missingpersonidentification')

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # Preparing SQL query to INSERT a record into the database.
    sql = "INSERT INTO findlog(name, location, typee, datee, timee) VALUES(%s, %s, %s, %s, %s)"
    
    now = datetime.now()
    datee = now.strftime("%d/%m/%Y")
    timee = now.strftime("%H:%M:%S")
    ftype = "Detected"

    try:
       # Executing the SQL command
       val = (fname, location, ftype, datee, timee)
       cursor.execute(sql, val)

       # Commit your changes in the database
       conn.commit()
    
    except:
       # Rolling back in case of error
       conn.rollback()
        
        
for name in os.listdir(known_faces_dir1):
    dir_path = os.path.join(known_faces_dir1, name)
    if os.path.isdir(dir_path):
        for filename in os.listdir(dir_path):
            if not filename.startswith('.'): 
                filepath = os.path.join(dir_path, filename)
                image= face_recognition.load_image_file(f"{known_faces_dir1}/{name}/{filename}")
                encoding=face_recognition.face_encodings(image)[0]
                known_faces.append(encoding)
                known_names.append(name)


# In[10]:

tempp="unknown"
video = cv2.VideoCapture(0)
while(True) :
    r,img = video.read()
    #locations = face_recognition.face_locations(img,model = "cnn")
    locations = face_recognition.face_locations(img,number_of_times_to_upsample=0,model = "cnn")
    encodings= face_recognition.face_encodings(img, locations)
    
    for face_encoding, face_location in zip(encodings,locations):
        results = face_recognition.compare_faces(known_faces,face_encoding,TOLERANCE)
        match=None
        if True in results:
            match=known_names[results.index(True)]
            cv2.rectangle(img,(face_location[3],face_location[0]),(face_location[1],face_location[2]),(0,255,0),3)
            cv2.putText(img,match,(face_location[3]+10,face_location[2]+15),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
            if tempp!=match:
                dataAdd(match)
                tempp=match
        else:
            cv2.rectangle(img,(face_location[3],face_location[0]),(face_location[1],face_location[2]),(0,255,0),3)
            cv2.putText(img,"Unknown",(face_location[3]+10,face_location[2]+15),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
            
    cv2.imshow('img',img)
    
    if cv2.waitKey(27)==ord('q'):
        break
    
video.release()
cv2.destroyAllWindows()
        
    

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
video.release()


 [INFO] Exiting Program and cleanup stuff
